In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
import numpy as np
from src.data.shape_data import fill_years

## Interpolate Undefined Values by Zip
This notebook groups the data by zipcode and then performs linear interpolation on each column to fill in undefined values.

In [43]:
filename = '../../data/interim/2.0-output.csv'
df = pd.read_csv(filename)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 944065 entries, 0 to 944064
Data columns (total 11 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   year         944065 non-null  int64  
 1   zipcode      944065 non-null  int64  
 2   EQI_zip      824770 non-null  float64
 3   SFR_zip      824770 non-null  float64
 4   RECPI_zip    824770 non-null  float64
 5   EQI_MSA      688518 non-null  float64
 6   SFR_MSA      688518 non-null  float64
 7   RECPI_MSA    688518 non-null  float64
 8   EQI_state    944065 non-null  float64
 9   SFR_state    944065 non-null  float64
 10  RECPI_state  944065 non-null  float64
dtypes: float64(9), int64(2)
memory usage: 79.2 MB
